In [ ]:
# default_exp exporters.exporters
%load_ext autoreload
%autoreload 2

In [ ]:
# export
# hide
from typing import Dict, List, Optional, Iterable, Any
import pandas as pd
import json

from pymemri.data.itembase import Item

# Query

With the `Query` class you can query the Pod with a list of properties, and export the result to a tabular dataformat. These properties can include edges that point to different items.

In [ ]:
# export
class Query:
    def __init__(self, *properties: List[str]):
        """
        A Query implements functionality to retrieve data from the pod to a tabular format.
        
        Given a list of `properties`, the `execute` method queries the pod for a set of given items, 
        and retrieves the properties for each item if it exists. Note that a properties can be nested behind
        multiple edges, such as "sender.owner.firstName".
        """
        self.properties = list(properties)

    def traverse_edges(self, client: "PodClient", items: List[Item], edges: List[str]) -> List[Item]:
        items = items.copy()

        for edge in edges:
            ids_to_query = list()
            query_item_idx = list()
            for i in range(len(items)):
                item = items[i]
                if item is None:
                    continue
                # Replace item with target item. If the edge is empty, it has to be queried again.
                try:
                    if edge not in item.edges:
                        items[i] = None
                    else:
                        items[i] = getattr(item, edge)[0]
                except Exception:
                    ids_to_query.append(item.id)
                    query_item_idx.append(i)
                    items[i] = None

            
            new_items = client.search({"ids": ids_to_query})
            for i, new_item in zip(query_item_idx, new_items):
                try:
                    items[i] = getattr(new_item, edge)[0]
                except Exception:
                    items[i] = None
        return items

    def get_property_values(
        self, client: "PodClient", prop: str, items: List[Item]
    ) -> list:
        edges, prop_name = self.parse_property(prop)
        target_items = self.traverse_edges(client, items, edges)

        result = [getattr(item, prop_name, None) for item in target_items]
        return result

    @staticmethod
    def parse_property(prop: str):
        prop = prop.split(".")
        edges = prop[:-1]
        prop = prop[-1]
        return edges, prop

    def convert_dtype(self, result, dtype):
        if dtype == "dict":
            return result
        elif dtype == "list":
            return [result[prop] for prop in self.properties]
        elif dtype in {"pandas", "pd", "df"}:
            return pd.DataFrame.from_dict(result)
        else:
            raise ValueError(f"Unknown dtype: {dtype}")

    def execute(self, client: "PodClient", items: List[Item], dtype="dict") -> Any:
        result = {
            prop: self.get_property_values(client, prop, items) for prop in self.properties
        }
        return self.convert_dtype(result, dtype)

## Tests

In [ ]:
# hide
from pymemri.pod.client import PodClient
from pymemri.data.schema import Account, Person, Message, CategoricalLabel
from pymemri.data.itembase import Edge
import random

### Create dummy data for dataset-

In [ ]:
# hide
client = PodClient()
client.add_to_schema(Account, Person, Message, CategoricalLabel)

num_items = 10
messages = []
items = []
edges = []
for i in range(num_items):   
    msg = Message(content=f"content_{i}", service="my_service")
    account = Account(handle=f"account_{i}")
    person = Person(firstName=f"firstname_{i}")
    label = CategoricalLabel(labelValue=f"label_{i}")
    items.extend([msg, account, person, label])
    edges.extend([
        Edge(msg, account, "sender"),
        Edge(msg, label, "label"),
        Edge(account, person, "owner")
    ])
    messages.append(msg)
    
# Dataset is not perfect, drop some random edges
edges = random.sample(edges, int(len(edges)*0.8))

client.bulk_action(
    create_items=items,
    create_edges=edges
)

### Example usage
For example, if we have a list of Messages and we want to get message content,
the accompanying account handles and account owner names, we can query:

In [ ]:
messages = client.search({"type": "Message", "service": "my_service"})
query = Query("content", "sender.handle", "sender.owner.firstName")
query.execute(client, messages, dtype="pandas")

In [ ]:
q = Query("content", "label.labelValue", "sender.owner.firstName", "sender.handle", "wrong_property")
result = q.execute(client, messages)
result

In [ ]:
# hide
q = Query("content", "label.labelValue", "sender.owner.firstName", "sender.handle", "wrong_property")
result = q.execute(client, messages)

assert all(len(vals) == len(result["content"]) for vals in result.values())
assert len(result["content"]) == num_items

# Check if columns all align
valid_props = ["label.labelValue", "sender.owner.firstName", "sender.handle"]
for i in range(num_items):
    row = [result[prop][i] for prop in valid_props]
    row_idx = [val[-1] for val in row if val is not None]
    assert len(set(row_idx)) <= 1

assert all(val is None for val in result["wrong_property"])

In [ ]:
q = Query("content", "label.name", "sender.owner.firstName", "sender.handle")
result = q.execute(client, messages, dtype="pandas")
result.head()

In [ ]:
# hide
from nbdev.export import *
notebook2script()